In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Coba coba Yolov8

In [ ]:
!pip install ultralytics roboflow opencv-python pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 132.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [ ]:
# Ekstrak dataset
import zipfile
import os

# Sesuaikan path ke file .zip Anda
zip_path = '/content/drive/MyDrive/Colab Notebooks/fracatlas.v3i.yolov11.zip'  # Ganti dengan path Anda

# Tentukan folder tujuan ekstrak
extract_path = '/content/drive/MyDrive/Colab Notebooks/fracatlas'
os.makedirs(extract_path, exist_ok=True)

# Ekstrak
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✓ Dataset berhasil diekstrak!")
print(f"Isi folder: {os.listdir(extract_path)}")

✓ Dataset berhasil diekstrak!
Isi folder: ['README.dataset.txt', 'README.roboflow.txt', 'data.yaml', 'test', 'train', 'valid']


In [ ]:
import os
import yaml
from pathlib import Path
import cv2
import numpy as np

# Tentukan lokasi data.yaml
yaml_path = "/content/drive/MyDrive/Colab Notebooks/fracatlas/data.yaml"  # Sesuaikan path jika berbeda

print("=" * 60)
print("VALIDASI STRUKTUR DATASET FRACATLAS")
print("=" * 60)

# 1. Cek apakah file YAML ada
if not os.path.exists(yaml_path):
    print(f"❌ File {yaml_path} tidak ditemukan!")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Files in current directory: {os.listdir('.')}")
else:
    print(f"✓ File YAML ditemukan: {yaml_path}")

    # 2. Baca YAML
    with open(yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)

    print("\n" + "=" * 60)
    print("KONFIGURASI DATASET:")
    print("=" * 60)
    print(f"Jumlah kelas: {data_config['nc']}")
    print(f"Nama kelas: {data_config['names']}")

    # 3. Validasi path
    print("\n" + "=" * 60)
    print("VALIDASI PATH FOLDER:")
    print("=" * 60)

    paths_to_check = {
        'train': data_config.get('train'),
        'val': data_config.get('val'),
        'test': data_config.get('test')
    }

    yaml_dir = os.path.dirname(os.path.abspath(yaml_path))
    print(f"Direktori YAML: {yaml_dir}\n")

    all_paths_valid = True
    dataset_stats = {}

    for split_name, rel_path in paths_to_check.items():
        if rel_path:
            # Resolve absolute path
            abs_path = os.path.join(yaml_dir, rel_path)
            abs_path_resolved = os.path.abspath(abs_path)
            exists = os.path.exists(abs_path_resolved)

            status = "✓" if exists else "❌"
            print(f"{status} {split_name.upper()}")
            print(f"   Relative path: {rel_path}")
            print(f"   Absolute path: {abs_path_resolved}")

            if exists:
                num_images = len([f for f in os.listdir(abs_path_resolved)
                                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                print(f"   Jumlah gambar: {num_images}")
                dataset_stats[split_name] = num_images
            else:
                all_paths_valid = False
                print(f"   ERROR: Path tidak ditemukan!")
            print()

    # 4. Cek struktur labels
    print("=" * 60)
    print("VALIDASI STRUKTUR ANOTASI:")
    print("=" * 60)

    if 'train' in paths_to_check:
        train_img_path = os.path.join(yaml_dir, paths_to_check['train'])
        train_img_abs = os.path.abspath(train_img_path)

        if os.path.exists(train_img_abs):
            # Asumsi labels ada di parent directory dengan nama 'labels'
            train_base = os.path.dirname(train_img_abs)
            labels_path = os.path.join(train_base, 'labels')

            print(f"Mencari labels di: {labels_path}")

            if os.path.exists(labels_path):
                num_labels = len([f for f in os.listdir(labels_path)
                                 if f.endswith('.txt')])
                print(f"✓ Folder labels ditemukan")
                print(f"  Jumlah file label (.txt): {num_labels}")

                # Cek format label
                label_files = [f for f in os.listdir(labels_path) if f.endswith('.txt')]
                if label_files:
                    sample_label = os.path.join(labels_path, label_files[0])
                    with open(sample_label, 'r') as f:
                        sample_content = f.readlines()
                    print(f"\n  Sample label file ({label_files[0]}):")
                    for i, line in enumerate(sample_content[:3]):
                        print(f"    Line {i+1}: {line.strip()}")
            else:
                print(f"⚠ Folder labels tidak ditemukan di: {labels_path}")
                print(f"  Silakan cek struktur folder dataset Anda")

    # 5. Rekomendasi
    print("\n" + "=" * 60)
    print("REKOMENDASI:")
    print("=" * 60)

    if all_paths_valid:
        print("✓ Struktur dataset valid!")
        print("\nUntuk training, gunakan:")
        print(f'  model.train(data="{yaml_path}", epochs=30, imgsz=640)')
    else:
        print("❌ Ada masalah dengan path. Silakan cek struktur folder:")
        print("\nStruktur yang diharapkan:")
        print("""
project_folder/
├── data.yaml
├── train/
│   ├── images/
│   └── labels/
├── valid/
│   ├── images/
│   └── labels/
└── test/
    ├── images/
    └── labels/
        """)

VALIDASI STRUKTUR DATASET FRACATLAS
✓ File YAML ditemukan: /content/drive/MyDrive/Colab Notebooks/fracatlas/data.yaml

KONFIGURASI DATASET:
Jumlah kelas: 6
Nama kelas: ['fractured', 'hand', 'hardware', 'hip', 'leg', 'shoulder']

VALIDASI PATH FOLDER:
Direktori YAML: /content/drive/MyDrive/Colab Notebooks/fracatlas

✓ TRAIN
   Relative path: ../train/images
   Absolute path: /content/drive/MyDrive/Colab Notebooks/train/images
   Jumlah gambar: 8280

✓ VAL
   Relative path: ../valid/images
   Absolute path: /content/drive/MyDrive/Colab Notebooks/valid/images
   Jumlah gambar: 789

✓ TEST
   Relative path: ../test/images
   Absolute path: /content/drive/MyDrive/Colab Notebooks/test/images
   Jumlah gambar: 394

VALIDASI STRUKTUR ANOTASI:
Mencari labels di: /content/drive/MyDrive/Colab Notebooks/train/labels
✓ Folder labels ditemukan
  Jumlah file label (.txt): 8280

  Sample label file (IMG0000001_jpg.rf.474a10fe47fb2b3cf56b877f1d4edb95.txt):
    Line 1: 4 0.73359375 0.4828125 0.5328125 0

In [ ]:
# Restart kernel & clean install
!pip uninstall ultralytics -y -q
!pip cache purge
!pip install ultralytics==8.0.195 opencv-python pyyaml -q

print("✓ Clean install completed")

Files removed: 62
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.9/618.9 kB 16.2 MB/s eta 0:00:00
✓ Clean install completed


In [10]:
!pip install --upgrade ultralytics -q
from ultralytics import YOLO
print("✓ Import successful")

✓ Import successful


In [ ]:
import subprocess
import sys
import os

print("=" * 70)
print("CLEAN INSTALL & TRAINING")
print("=" * 70)

# ============================================================
# 0. NUCLEAR CLEAN
# ============================================================
print("\n[0/4] Performing clean install...")

subprocess.run([sys.executable, '-m', 'pip', 'uninstall', 'ultralytics', '-y', '-q'],
               stderr=subprocess.DEVNULL)
subprocess.run([sys.executable, '-m', 'pip', 'cache', 'purge'],
               stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Install latest stable version
subprocess.run([sys.executable, '-m', 'pip', 'install',
                'ultralytics', '-q'],
               stdout=subprocess.DEVNULL)

print("✓ Clean install completed\n")

# ============================================================
# 1. IMPORT & VERIFY
# ============================================================
print("[1/4] Importing and verifying...")

from ultralytics import YOLO
import torch
import os

print(f"PyTorch: {torch.__version__}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print("✓ Ready to train\n")

# ============================================================
# 2. DATASET PATH
# ============================================================
print("[2/4] Setting up dataset...")

yaml_path = "/content/fracatlas_dataset/data.yaml"

# Verifikasi
if not os.path.exists(yaml_path):
    print(f"❌ YAML tidak ditemukan. Searching...")
    for root, dirs, files in os.walk("/content/fracatlas_dataset"):
        for file in files:
            if file == "data.yaml":
                yaml_path = os.path.join(root, file)
                print(f"✓ Found: {yaml_path}")
                break

print(f"Dataset: {yaml_path}\n")

# ============================================================
# 3. LOAD MODEL
# ============================================================
print("[3/4] Loading YOLOv8s...")
model = YOLO('yolov8s.pt')
print("✓ Model loaded\n")

# ============================================================
# 4. TRAINING
# ============================================================
print("[4/4] Starting training...")
print("=" * 70)

# Simple config tanpa callbacks yang bermasalah
results = model.train(
    data=yaml_path,
    epochs=3,
    imgsz=640,
    batch=16,
    device=0,
    workers=4,
    patience=10,
    save=True,
    save_period=5,
    project='/content/drive/MyDrive/Colab Notebooks/fracatlas/runs',
    name='yolov8s_baseline',
    exist_ok=False,
    verbose=True,
    # Disable problematic features
    nms=False,
    profile=False,
    plots=True,
)

print("=" * 70)
print("\n✓ TRAINING SELESAI!\n")

# ============================================================
# RESULTS
# ============================================================
print("HASIL:")
print("=" * 70)
print(f"✓ Model tersimpan di Google Drive")
print(f"✓ Check: /content/drive/MyDrive/Colab Notebooks/fracatlas/runs/yolov8s_baseline/")
print("=" * 70)

CLEAN INSTALL & TRAINING

[0/4] Performing clean install...
✓ Clean install completed

[1/4] Importing and verifying...
PyTorch: 2.9.0+cu126
GPU: Tesla T4
✓ Ready to train

[2/4] Setting up dataset...
Dataset: /content/fracatlas_dataset/data.yaml

[3/4] Loading YOLOv8s...
✓ Model loaded

[4/4] Starting training...
Ultralytics 8.3.230 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/fracatlas_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=3, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7

Pakai mirror huggingface/langsung dari ultralytics nya

# Yolov11

In [1]:
!pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.1/112.6 GB disk)


In [5]:
model = YOLO('yolo11n.pt') # Pre-trained model

In [11]:
model.train(
   data='/content/drive/MyDrive/Colab Notebooks/dataset/data.yaml',
   epochs=50,    # Set
   patience=10,   # Berhenti jika tidak ada perbaikan selama 10 epoch
   imgsz=640,
   batch=16
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Colab Notebooks/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspec

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x79e3578c24b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [14]:
# Lihat best weights
import os
from pathlib import Path

runs_dir = Path('/content/runs/detect/train')
if runs_dir.exists():
    print("Training results:")
    for item in runs_dir.iterdir():
        if item.is_dir():
            print(f"  - {item.name}")
            weights_dir = item / 'weights'
            if weights_dir.exists():
                for w in weights_dir.glob('*.pt'):
                    size = w.stat().st_size / (1024*1024)
                    print(f"    • {w.name} ({size:.1f}MB)")

Training results:
  - weights


In [13]:
from google.colab import files
import shutil

# 1. Zip folder 'runs'
shutil.make_archive('/content/hasil_training', 'zip', '/content/runs')

# 2. Download file zip
files.download('/content/hasil_training.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Coba coba aja

In [ ]:
import os
import sys
import urllib.request
from datetime import datetime

print("=" * 70)
print("YOLOV11 TRAINING VIA HUGGING FACE MIRROR")
print("=" * 70)

# ============================================================
# 0. DOWNLOAD MODEL FROM HUGGING FACE
# ============================================================
print("\n[0/4] Downloading YOLOv11s from Hugging Face...")

model_url = "https://huggingface.co/Ultralytics/YOLOv11/resolve/main/yolov11s.pt"
model_path = "yolov11s.pt"

try:
    print(f"Downloading from: {model_url}")
    urllib.request.urlretrieve(model_url, model_path)
    print(f"✓ Model downloaded: {model_path}\n")
except Exception as e:
    print(f"❌ Download failed: {e}")
    print("Trying alternative mirror...")

    # Try alternative URL
    alt_urls = [
        "https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov11s.pt",
        "https://cdn-lfs.huggingface.net/repos/Ultralytics/YOLOv11/main/yolov11s.pt"
    ]

    for alt_url in alt_urls:
        try:
            print(f"Trying: {alt_url}")
            urllib.request.urlretrieve(alt_url, model_path)
            print(f"✓ Model downloaded from alternative source\n")
            break
        except:
            continue
    else:
        print("❌ All downloads failed. Exiting.")
        sys.exit(1)

# Verify file exists and size
if os.path.exists(model_path):
    size_mb = os.path.getsize(model_path) / (1024 * 1024)
    print(f"✓ File verified: {size_mb:.1f} MB\n")
else:
    print("❌ Model file not found after download")
    sys.exit(1)

# ============================================================
# 1. IMPORT ULTRALYTICS
# ============================================================
print("[1/4] Importing Ultralytics...")

try:
    from ultralytics import YOLO
    import torch
    print(f"PyTorch: {torch.__version__}")
    print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
    print("✓ Import successful\n")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Installing ultralytics...")
    os.system('pip install ultralytics -q')
    from ultralytics import YOLO
    import torch
    print("✓ Ultralytics installed and imported\n")

# ============================================================
# 2. LOAD MODEL
# ============================================================
print("[2/4] Loading YOLOv11s model...")

try:
    model = YOLO(model_path)
    print("✓ YOLOv11s model loaded\n")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    sys.exit(1)

# ============================================================
# 3. PREPARE DATASET
# ============================================================
print("[3/4] Preparing dataset...")

yaml_path = "/content/fracatlas_dataset/data.yaml"

if not os.path.exists(yaml_path):
    print(f"Searching for data.yaml...")
    for root, dirs, files in os.walk("/content/fracatlas_dataset"):
        for file in files:
            if file == "data.yaml":
                yaml_path = os.path.join(root, file)
                print(f"✓ Found: {yaml_path}")
                break

print(f"Dataset: {yaml_path}\n")

# ============================================================
# 4. TRAINING
# ============================================================
print("[4/4] Starting YOLOv11s training...")
print("=" * 70)

try:
    results = model.train(
        data=yaml_path,
        epochs=3,
        imgsz=640,
        batch=16,
        device=0,
        workers=4,
        patience=10,
        save=True,
        save_period=5,
        project='/content/drive/MyDrive/Colab Notebooks/fracatlas/runs',
        name=f'yolov11s_baseline_{datetime.now().strftime("%Y%m%d_%H%M%S")}',
        exist_ok=False,
        verbose=True,
        nms=False,
        profile=False,
    )

    print("=" * 70)
    print("\n✓ TRAINING COMPLETED!\n")

except Exception as e:
    print(f"\n❌ Training error: {e}")
    print("Attempting to continue...")
    import traceback
    traceback.print_exc()

# ============================================================
# RESULTS
# ============================================================
print("HASIL TRAINING:")
print("=" * 70)
print("✓ Model tersimpan di Google Drive")
print("✓ Check: /content/drive/MyDrive/Colab Notebooks/fracatlas/runs/")
print("=" * 70)

YOLOV11 TRAINING VIA HUGGING FACE MIRROR

[0/4] Downloading YOLOv11s from Hugging Face...
❌ Download failed: HTTP Error 401: Unauthorized
Trying alternative mirror...
Trying: https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov11s.pt


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Trying: https://cdn-lfs.huggingface.net/repos/Ultralytics/YOLOv11/main/yolov11s.pt
❌ All downloads failed. Exiting.
Traceback (most recent call last):
  File "/tmp/ipython-input-1130496337.py", line 20, in <cell line: 0>
    urllib.request.urlretrieve(model_url, model_path)
  File "/usr/lib/python3.12/urllib/request.py", line 240, in urlretrieve
    with contextlib.closing(urlopen(url, data)) as fp:
                            ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/urllib/request.py", line 215, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/urllib/request.py", line 521, in open
    response = meth(req, response)
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/urllib/request.py", line 630, in http_response
    response = self.parent.error(
               ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/urllib/request.py", line 559, in error
    return self._call_chain(*args)
           ^^^

TypeError: object of type 'NoneType' has no len()